
# Import Python libraries.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import Datasets

In [ ]:
train_data = pd.read_csv(r"/content/drive/MyDrive/Training Data.csv")
# test_data = pd.read_csv(r"E:\Problem statement\Dataset\test1_df.csv")
train_data.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


# Convert datatype of selected fields.

In [ ]:
dfxtrain = train_data.drop(labels=['risk_flag', 'Id',],axis=1)
dfytrain=train_data['risk_flag']
dfxtrain= pd.get_dummies(dfxtrain,drop_first=True)
dfxtrain

,income,age,experience,current_job_years,current_house_years,married_single,house_ownership_owned,house_ownership_rented,car_ownership_yes,profession_Analyst,profession_Architect,profession_Army_officer,profession_Artist,profession_Aviator,profession_Biomedical_Engineer,profession_Chartered_Accountant,profession_Chef,profession_Chemical_engineer,profession_Civil_engineer,profession_Civil_servant,profession_Comedian,profession_Computer_hardware_engineer,profession_Computer_operator,profession_Consultant,profession_Dentist,profession_Design_Engineer,profession_Designer,profession_Drafter,profession_Economist,profession_Engineer,profession_Fashion_Designer,profession_Financial_Analyst,profession_Firefighter,profession_Flight_attendant,profession_Geologist,profession_Graphic_Designer,profession_Hotel_Manager,profession_Industrial_Engineer,profession_Lawyer,profession_Librarian,...,city_Ulhasnagar,city_Uluberia,city_Unnao,city_Vadodara,city_Varanasi,city_Vasai-Virar,city_Vellore,city_Vijayanagaram,city_Vijayawada,city_Visakhapatnam[4],city_Warangal[11][12],city_Yamunanagar,state_Assam,state_Bihar,state_Chandigarh,state_Chhattisgarh,state_Delhi,state_Gujarat,state_Haryana,state_Himachal_Pradesh,state_Jammu_and_Kashmir,state_Jharkhand,state_Karnataka,state_Kerala,state_Madhya_Pradesh,state_Maharashtra,state_Manipur,state_Mizoram,state_Odisha,state_Puducherry,state_Punjab,state_Rajasthan,state_Sikkim,state_Tamil_Nadu,state_Telangana,state_Tripura,state_Uttar_Pradesh,state_Uttar_Pradesh[5],state_Uttarakhand,state_West_Bengal
0,1303835,23,3,3,13,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7574516,40,10,9,13,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3991815,66,4,4,10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6256451,41,2,2,12,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5768871,47,11,3,14,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,8154883,43,13,6,11,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
251996,2843572,26,10,6,11,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
251997,4522448,46,7,7,12,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
251998,6507128,45,0,0,10,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
from imblearn.over_sampling import SMOTE
ss=SMOTE()
dfxtrain, dfytrain=ss.fit_resample(dfxtrain, dfytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


# Drop the dependent variable from the train/test dataset. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_x = scaler.fit_transform(dfxtrain)
# xs_train, xs_test, ys_train, ys_test = train_test_split(scaled_x, ytrain, test_size=0.20,shuffle=True)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_x, dfytrain, test_size=0.20,shuffle=True)

In [ ]:
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
#Using Logistic Regression 
clf = NuSVC()
clf.fit(x_train, y_train)
clf.score(x_test,y_test)

In [ ]:
print(x_train.shape,'  ',y_train.shape)
print(x_test.shape,'  ',y_test.shape)

NameError: ignored

# Train your model

In [ ]:
from xgboost import XGBClassifier
eval_set = [(dfxtrain,dfytrain), (x_tes, y_tes)]
eval_metric = ["auc","error"]
# dfxtrain.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in dfxtrain.columns.values]
# x_tes.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_tes.columns.values]
model = XGBClassifier(silent=False, 
                      learning_rate=0.001,  
                      colsample_bytree = 0.8,
                      subsample = 1,
                      objective='binary:logistic', 
                      n_estimators=100, 
                      reg_alpha = 0.1,
                      max_depth=50, 
                      gamma=10,
                      tree_method='gpu_hist')

%time res=model.fit(dfxtrain,dfytrain, eval_metric=eval_metric, eval_set=eval_set, verbose=True)


[0]	validation_0-auc:0.91327	validation_0-error:0.145337	validation_1-auc:0.902056	validation_1-error:0.155313
[1]	validation_0-auc:0.92943	validation_0-error:0.112145	validation_1-auc:0.919769	validation_1-error:0.120789
[2]	validation_0-auc:0.932702	validation_0-error:0.107592	validation_1-auc:0.924816	validation_1-error:0.114907
[3]	validation_0-auc:0.93999	validation_0-error:0.097216	validation_1-auc:0.932037	validation_1-error:0.106084
[4]	validation_0-auc:0.940822	validation_0-error:0.104636	validation_1-auc:0.933583	validation_1-error:0.113176
[5]	validation_0-auc:0.941321	validation_0-error:0.103949	validation_1-auc:0.933951	validation_1-error:0.112203
[6]	validation_0-auc:0.941735	validation_0-error:0.102196	validation_1-auc:0.934709	validation_1-error:0.109307
[7]	validation_0-auc:0.941577	validation_0-error:0.102555	validation_1-auc:0.934536	validation_1-error:0.109556
[8]	validation_0-auc:0.943457	validation_0-error:0.099139	validation_1-auc:0.936445	validation_1-error:0.10

In [ ]:
import matplotlib.pyplot as plt
plt.plot(res)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
nn = RandomForestClassifier().fit(scaled_x,dfytrain)
# print(nn.score(x1tr,y1tr))
# print(nn.score(x_tes, y_tes))

In [ ]:
nn1 = SVC(gamma='auto').fit(dfxtrain,dfytrain)

# Check your roc_auc_score 

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred=nn.predict(x_tes)
# labels=[int(x[0]) for x in y_pred]
auc = roc_auc_score(y_pred, y_tes)
print('ROC AUC: %f' % auc)
y_pred

ROC AUC: 0.936825


array([0, 1, 1, ..., 1, 0, 0])

#load test data


In [ ]:
test_data = pd.read_csv(r"/content/drive/MyDrive/Test Data.csv")
# test_data = pd.read_csv(r"E:\Problem statement\Dataset\test1_df.csv")|

In [ ]:
dfxtr = test_data.drop(labels=['id'],axis=1)
dfxtr= pd.get_dummies(dfxtr,drop_first=True)
dfxtr

,income,age,experience,current_job_years,current_house_years,married_single,house_ownership_owned,house_ownership_rented,car_ownership_yes,profession_Analyst,profession_Architect,profession_Army officer,profession_Artist,profession_Aviator,profession_Biomedical Engineer,profession_Chartered Accountant,profession_Chef,profession_Chemical engineer,profession_Civil engineer,profession_Civil servant,profession_Comedian,profession_Computer hardware engineer,profession_Computer operator,profession_Consultant,profession_Dentist,profession_Design Engineer,profession_Designer,profession_Drafter,profession_Economist,profession_Engineer,profession_Fashion Designer,profession_Financial Analyst,profession_Firefighter,profession_Flight attendant,profession_Geologist,profession_Graphic Designer,profession_Hotel Manager,profession_Industrial Engineer,profession_Lawyer,profession_Librarian,...,city_Ulhasnagar,city_Uluberia,city_Unnao,city_Vadodara,city_Varanasi,city_Vasai-Virar,city_Vellore,city_Vijayanagaram,city_Vijayawada,city_Visakhapatnam[4],city_Warangal[11][12],city_Yamunanagar,state_Assam,state_Bihar,state_Chandigarh,state_Chhattisgarh,state_Delhi,state_Gujarat,state_Haryana,state_Himachal Pradesh,state_Jammu and Kashmir,state_Jharkhand,state_Karnataka,state_Kerala,state_Madhya Pradesh,state_Maharashtra,state_Manipur,state_Mizoram,state_Odisha,state_Puducherry,state_Punjab,state_Rajasthan,state_Sikkim,state_Tamil Nadu,state_Telangana,state_Tripura,state_Uttar Pradesh,state_Uttar Pradesh[5],state_Uttarakhand,state_West Bengal
0,7393090,59,19,4,13,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1215004,25,5,5,10,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8901342,50,12,9,14,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1944421,49,9,3,12,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,13429,25,18,13,11,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,9955481,57,13,5,10,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27996,2917765,47,9,9,14,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27997,8082415,24,5,4,13,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27998,9474180,51,13,13,14,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# processing test data

# Drop the dependent variable from the train/test dataset. 

In [ ]:
dfxtr = scaler.fit_transform(dfxtr)

In [ ]:
y_pred=nn.predict(dfxtr)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
test_data['pred']=y_pred
test_data

,id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,pred
0,1,7393090,59,19,single,rented,no,Geologist,Malda,West Bengal,4,13,0
1,2,1215004,25,5,single,rented,no,Firefighter,Jalna,Maharashtra,5,10,0
2,3,8901342,50,12,single,rented,no,Lawyer,Thane,Maharashtra,9,14,0
3,4,1944421,49,9,married,rented,yes,Analyst,Latur,Maharashtra,3,12,0
4,5,13429,25,18,single,rented,yes,Comedian,Berhampore,West Bengal,13,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,27996,9955481,57,13,single,rented,no,Statistician,Eluru[25],Andhra Pradesh,5,10,0
27996,27997,2917765,47,9,single,rented,no,Technical writer,Ratlam,Madhya Pradesh,9,14,0
27997,27998,8082415,24,5,single,rented,no,Lawyer,Mira-Bhayandar,Maharashtra,4,13,0
27998,27999,9474180,51,13,single,rented,yes,Chartered Accountant,Bhilai,Chhattisgarh,13,14,0


In [ ]:
to_csv_maker=test_data[['id','pred']]
to_csv_maker=to_csv_maker.rename(columns={"id": "id", "pred": "risk_flag"})
to_csv_maker

,id,risk_flag
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [ ]:
to_csv_maker.to_csv('submission.csv',index=False)

In [ ]:
to_csv_maker['risk_flag'].value_counts().to_frame().T

,0,1
risk_flag,24957,3043


In [ ]:
test_data = pd.read_csv(r"/content/potential to be best.csv")
test_data['risk_flag'].value_counts().to_frame().T

,0,1
risk_flag,22626,5374


In [ ]:
test_data = pd.read_csv(r"/content/submission.csv")
test_data['risk_flag'].value_counts().to_frame().T

,0,1
risk_flag,24960,3040


In [ ]:
test_data = pd.read_csv(r"/content/best till now85_13.csv")
test_data['risk_flag'].value_counts().to_frame().T

,0,1
risk_flag,22808,5192
